<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 1

import os,sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

import SimpleITK as sitk
from scipy import ndimage as ndi
import skimage as skim
from skimage import feature
import pandas as pd
import xarray as xr
plt.rcParams['figure.figsize'] = (10,8)
import pickle as pkl
from tensorboardX import SummaryWriter
from torch.utils.data import Dataset, DataLoader
import torchvision.utils
from torchsummary import summary
import torch
import torch.nn as nn
from collections import defaultdict
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import models
import time
import copy

%aimport mre 
from mre import pytorch_unet_tb
%aimport mre.mre_datasets
from mre.mre_datasets import MRETorchDataset, MREtoXr
%aimport mre.plotting
from mre.plotting import torch_dl_vis, hv_dl_vis, hv_pred_comp
%aimport mre.train_model_full_v2
from mre.train_model_full_v2 import train_model_full

import holoviews as hv
from holoviews import opts
hv.extension('bokeh')


# Load mre data
#xr_maker = MREtoXr(from_file='/pghbio/dbmi/batmanlab/Data/MRE/XR/*.nc')
#ds = xr_maker.get_ds()
#ds.persist()
#out_dir = '/pghbio/dbmi/batmanlab/bpollack/predictElasticity/data'


In [ ]:
data_path = '/pghbio/dbmi/batmanlab/Data/MRE/XR_trimmed/'
data_file = '*.nc'
output_path = '/pghbio/dbmi/batmanlab/bpollack/predictElasticity/data'
torch.cuda.empty_cache()
now = datetime.today().strftime('%Y-%m-%d_%H-%M-%S')
print(now)
kwargs = dict(seed=100, step_size=100, num_epochs=50, lr=0.01, in_channels=5, n_layers=7, channel_growth=True, model_cap=16)
inputs, targets, masks, names, model = train_model_full(data_path, data_file, output_path, model_version=f'notebook_test_{now}', dry_run=False, **kwargs)


In [4]:
model_pred = model(inputs)
hv_dl_vis(inputs, targets, masks, names, model_pred)
#hv_dl_vis(inputs, targets, masks, names)

Column
    [0] FloatSlider(end=1, name='mask', start=0, value=0.0)
    [1] Row
        [0] HoloViews(Layout)
        [1] Column
            [0] WidgetBox(css_classes=['widget-box'])
                [0] Select(margin=(20, 20, 20, 20), name='subject', options=['0492_0', '0492_1', ...], value='0492_0', width=250)
            [1] VSpacer()

In [ ]:
test_ds = add_LOO_predictions(ds, out_dir+'/trained_models/', version='2019-05-30_11-53-53_n0', coord_conv=False, cap=64, mask_trimmer=False, mask_mixer='union')

In [ ]:
pkl.dump(test_ds, open(full_data_dir+'mre_ds_preprocess_5_pred_v4.p', 'wb'), protocol=-1)

In [ ]:
hv.save(layout, 'filter_examples.png')

In [ ]:
model_trans = models.resnet50(pretrained=True).to('cuda')

In [ ]:
summary(model_trans, (3, 256, 256))

In [ ]:
model_trans = models.resnet18(pretrained=True).to('cuda')
class UNet_Transfer(nn.Module):
    def __init__(self, n_class, cap=16, coord_conv=True, input_model=model_trans):
        super().__init__()
        self.transfer_layers = nn.Sequential(*list(model_trans.children())[0:5])

    def forward(self, x):
        x = self.transfer_layers(x)
        # x = self.classifier(x)
        return x

In [ ]:
model_trans =UNet_Transfer(1).to('cuda')
print(model_trans)

In [ ]:
summary(model_trans, (3,256,256))